Задача поиск похожих по эмбеддингам
Скачиваем датасет (источник): положительные, отрицательные.
или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать 

1. объединить в одну выборку

In [54]:
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec, FastText

from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximdoroshenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [11]:
df = df.drop(columns=['label'])

In [12]:
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [4]:
!pip install gensim

In [5]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [19]:
custom_stopwords = set(stopwords.words("russian"))
df["tokens"] = df["text"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))

# Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values and keep relevant columns
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]

docs = df["text"].values
tokenized_docs = df["tokens"].values

In [15]:
df.head()

,text,tokens
43269,"@00000Asrova кто знает... :D А вообще, это кто...","[00000asrova, кто, знает, вообще, это, кто, был]"
23246,00:00.....пойти уроки что ль поделать..)))))))...,"[0000пойти, уроки, что, ль, поделать, httptcop..."
45781,@000Nana000 заправил систему кипяточком))) и в...,"[000nana000, заправил, систему, кипяточком, вп..."
7460,@000Nana000 Конечно! Зачем нам неудачники ))) ...,"[000nana000, конечно, зачем, нам, неудачники, ..."
24326,"@009_panda да надо скоро идти, но все никак не...","[009panda, да, надо, скоро, идти, но, все, ник..."


### Word2Vec

In [20]:
model = Word2Vec(sentences=tokenized_docs, vector_size=100, workers=1, seed=SEED)

In [21]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(tokenized_docs, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(217309, 100)

In [23]:
test_text = 'заправил машину кипяточком'
test_tokenized = clean_text(test_text, word_tokenize, custom_stopwords)
test_vec = vectorize([test_tokenized], model=model)
test_vec

[array([ 0.30283317, -0.20712417, -0.3462995 , -0.06517202, -0.3404316 ,
         0.6041342 ,  0.43395656,  0.42605087,  0.00391609, -0.2760664 ,
        -0.50516784, -0.23609805,  0.07685451,  0.5344544 , -0.13421503,
        -0.07629161,  0.11465648, -0.691084  , -0.40536916,  0.09282611,
        -0.04479497,  0.47768867, -0.01528123,  0.44763947,  0.28622422,
         0.28387883, -0.26889616,  0.16918483, -0.19874267, -0.24481964,
         0.00242973, -0.32306582,  0.37143135,  0.32551086,  0.05335397,
         0.26398084,  0.33884817, -0.9141328 , -0.31157762,  0.01555198,
         0.3320907 ,  0.07094458,  0.66456115, -0.06313147,  0.23505297,
         0.1222454 ,  0.24011584,  0.16154976,  0.5486938 , -0.6217685 ,
         0.14902207, -0.83998555,  0.18760987,  0.2313182 ,  0.4972492 ,
        -0.04477295, -0.48122576,  0.20603289,  0.14241582,  0.5108996 ,
        -0.17002346, -0.16331035, -0.47614282,  0.27799818,  0.17010558,
         0.14264111,  0.09437311, -0.29626024, -0.1

In [28]:
model.wv.similar_by_vector(test_vec[0], topn=5, restrict_vocab=None)

[('машину', 1.0),
 ('вокзал', 0.9892905950546265),
 ('рабочую', 0.9890004396438599),
 ('квартиру', 0.9887105822563171),
 ('мексике', 0.9887104034423828)]

In [44]:
from sklearn.neighbors import KDTree
X = np.vstack(vectorized_docs)
tree = KDTree(X)

In [51]:
_, ind = tree.query(test_vec, k=5)
[df.text.iloc[i] for i in ind[0]]

['Ого!!!бачила тікишо на обочині траси згорівшу машину(',
 'обида.(\nбилеты на елку не дают больше...(((\nбольшая стала:3',
 'в магазине нет редбулла((( зачем вышел? #жизньболь',
 'Тюменская челядь в очереди за халявкой))) http://t.co/WiL58G1rJv',
 '@igor_l33t 8  Тысяч то хватило?) и всех на кино пустили?:D']

In [52]:
test_text = 'в магазине нет редбулла((( зачем вышел? #жизньболь'
test_tokenized = clean_text(test_text, word_tokenize, custom_stopwords)
test_vec = vectorize([test_tokenized], model=model)

_, ind = tree.query(test_vec, k=10)
[df.text.iloc[i] for i in ind[0]]

['в магазине нет редбулла((( зачем вышел? #жизньболь',
 'Убираться в квартире за новые наушники вполне норм)',
 'Оф,еще кто то второго числа ходит за игрушками((бля',
 'пришла на кухню именно в ту минуту,когда по телеку шла реклама шуб за 1488 рублей в месяц. неонацисты рекламу теперь у нас делают?)',
 'обида.(\nбилеты на елку не дают больше...(((\nбольшая стала:3',
 'Опять потеряла где-то в доме документы на машину. Офигеть я клуша((',
 'Решили сходить в магазин, как оказалась не один в округе не работает и тут мы поняли, что даже магазины против нас.жизньболь(',
 '@r3nya гугль решил только 16Гб мимо плейстора продавать :(',
 '@Evgeniya996 реклама была и сказали что 12 числа будет развязка)',
 'Еще не успели собрать ёлку на площади, а она уже упала)))']

### FastText

In [55]:
modelFT = FastText(sentences=tokenized_docs, vector_size=300, min_count=1, window=5, workers=8)

In [58]:
vectorized_docs = vectorize(tokenized_docs, model=modelFT)
X = np.vstack(vectorized_docs)
tree = KDTree(X)

test_text = 'в магазине нет редбулла((( зачем вышел? #жизньболь'
test_tokenized = clean_text(test_text, word_tokenize, custom_stopwords)
test_vec = vectorize([test_tokenized], model=modelFT)

_, ind = tree.query(test_vec, k=10)
[df.text.iloc[i] for i in ind[0]]

['в магазине нет редбулла((( зачем вышел? #жизньболь',
 'И где этот   : Я уже дебиан установила! :(',
 'Ездили в триумф и в хэппимол - купили только одну юбку;(жизнь боль',
 'Хершеля убил Филипп, сижу и плачу. Зато Филиппа убили и, может, Джудит, маленькая, умерла. Одни разочарования :((  \n#TWD',
 '@chernyatkina я заебался у кабинета стоять))у тебя мама у кабинета стоит)',
 '@Tritatushe4ka моя симка после смерти мужа вышла замуж за робота семьи.. но от горя стала шлюхой и изменяет ему=((',
 'я выключил вкладку в которой играла музыка:((((9\nсук блеа',
 'Кашель заипал...уже и на грелках лежала и картошку клали :(',
 'Ездила в центр переливания крови...решилась все-таки,сдала свою третью группу))',
 'Айфон еле восстановил чуть с ума не сошел(']